# 1. Treinamento SVM

In [1]:
# Caso ainda não tenha instalado,
# !pip install tqdm
# !pip install tqdm_joblib

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC

# Importando as bibliotecas para paralelização e progress bar
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib

c:\Users\Ruy\anaconda3\Lib\site-packages\tqdm_joblib\__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Carregar os dados tratados
df = pd.read_csv("../Data/Stroke.csv")

# Separar X e y
X = df.drop('stroke', axis=1)
y = df['stroke']

In [3]:
# 7. Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
# 8. Configuração do Grid Search para otimizar o SVM
svm_model = SVC(probability=True)

# Definindo o grid de hiperparâmetros
param_grid = {
    'kernel': ['rbf', 'linear'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

# Configurando o GridSearchCV com validação cruzada de 5 folds e paralelização (n_jobs=-1)
grid_search = GridSearchCV(
    estimator=svm_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=0  # O tqdm exibirá o progresso, então podemos deixar verbose=0
)

In [5]:
# Configurando a Barra de progresso:
# Calculando o total de iterações: número de combinações * número de folds
total_iter = grid_search.cv * np.prod([len(v) for v in param_grid.values()])
print(f"Total de iterações estimadas: {total_iter}")

Total de iterações estimadas: 60


In [6]:
# 9. Realizando o Grid Search com tqdm_joblib para monitorar o progresso
with tqdm_joblib(tqdm(desc="GridSearchCV", total=total_iter)) as progress_bar:
    grid_search.fit(X_train, y_train)

GridSearchCV:   0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

In [7]:
print("Melhores hiperparâmetros:", grid_search.best_params_)
print("Melhor acurácia:", grid_search.best_score_)

Melhores hiperparâmetros: {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Melhor acurácia: 0.9808505870402101


In [8]:
# 10. Avaliação do modelo com os melhores hiperparâmetros
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
print("\nMatriz de Confusão:\n", confusion_matrix(y_test, y_pred))

# Caso deseje obter as probabilidades das predições
y_pred_proba = best_model.predict_proba(X_test)
print("Shape das predições de probabilidade:", y_pred_proba.shape)


Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      0.97      0.98       951
           1       0.97      1.00      0.98       943

    accuracy                           0.98      1894
   macro avg       0.98      0.98      0.98      1894
weighted avg       0.98      0.98      0.98      1894


Matriz de Confusão:
 [[918  33]
 [  0 943]]
Shape das predições de probabilidade: (1894, 2)
